## How many orders were done under each payment type?

In [5]:
query = """
SELECT 
	payment_type ,
	count( order_id )
FROM payments p 
GROUP BY payment_type 
"""

table = pd.read_sql_query( query, conn )
table

,payment_type,count( order_id )
0,boleto,19784
1,credit_card,76795
2,debit_card,1529
3,not_defined,3
4,voucher,5775


**_Fun thing to notice is the difference in values for the query below._**

In [19]:
query = """
SELECT 
	payment_type ,
	count( DISTINCT order_id )
FROM payments p 
GROUP BY payment_type 
"""

table = pd.read_sql_query( query, conn )
table

,payment_type,count( DISTINCT order_id )
0,boleto,19784
1,credit_card,76505
2,debit_card,1528
3,not_defined,3
4,voucher,3866


**_Let's explore that a bit:_**

In [3]:
# Capturing the order_ids that appear in repetition 

subquery = """
SELECT 
    order_id , 
    count( order_id ) AS num_repeat
FROM payments p 
GROUP BY order_id 
HAVING num_repeat > 1
ORDER BY num_repeat DESC
"""

pd.read_sql_query( subquery, conn )

,order_id,num_repeat
0,fa65dad1b0e818e3ccc5cb0e39231352,29
1,ccf804e764ed5650cd8759557269dc13,26
2,285c2e15bebd4ac83635ccc563dc71f4,22
3,895ab968e7bb0d5659d16cd74cd1650c,21
4,fedcd9f7ccdc8cba3a18defedd1a5547,19
...,...,...
2956,00e6bc6b166eb28b4502c1cad4457248,2
2957,00b4a910f64f24dbcac04fe54088a443,2
2958,0071ee2429bc1efdc43aa3e073a5290e,2
2959,002f19a65a2ddd70a090297872e6d64e,2


In [7]:
# Part of the payments table respective to the repeated order_ids (under JOIN with the subquery above)
query = """
SELECT p2.*, b.num_repeat
FROM payments p2
	LEFT JOIN 
	(
		SELECT order_id , count( order_id ) AS num_repeat
		FROM payments p 
		GROUP BY order_id 
		HAVING num_repeat > 1 
	) b
ON b.order_id = p2.order_id 
WHERE b.num_repeat >= 2 
ORDER BY p2.order_id, p2.payment_sequential 
"""

pd.read_sql_query( query, conn )

,order_id,payment_sequential,payment_type,payment_installments,payment_value,num_repeat
0,0016dfedd97fc2950e388d2971d718c7,1.0,credit_card,5,52.63,2
1,0016dfedd97fc2950e388d2971d718c7,2.0,voucher,1,17.92,2
2,002f19a65a2ddd70a090297872e6d64e,1.0,voucher,1,44.11,2
3,002f19a65a2ddd70a090297872e6d64e,2.0,voucher,1,33.18,2
4,0071ee2429bc1efdc43aa3e073a5290e,1.0,voucher,1,100.00,2
...,...,...,...,...,...,...
7402,ffa1dd97810de91a03abd7bd76d2fed1,2.0,voucher,1,418.73,2
7403,ffa39020fe7c8a3e907320e1bec4b985,1.0,credit_card,1,7.13,2
7404,ffa39020fe7c8a3e907320e1bec4b985,2.0,voucher,1,64.01,2
7405,ffc730a0615d28ec19f9cad02cb41442,1.0,credit_card,1,14.76,2


**_Thus, repeated order_ids seem to occur when more than one type of payment is used (different vouchers included)_**

## What are the maximum and minimum numbers of installments?

In [5]:
query = """
SELECT 
	max( payment_installments ),
	min( payment_installments )
FROM payments p
"""

table = pd.read_sql_query( query, conn )
table

,max( payment_installments ),min( payment_installments )
0,24,0


**_It seems a bit weird to have zero installments, but we can try to check this as below_**

In [8]:
query = """
SELECT *
FROM orders o LEFT JOIN payments p ON (p.order_id=o.order_id)
WHERE p.payment_installments == 0
"""

table = pd.read_sql_query( query, conn )
table

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,744bade1fcf9ff3f31d860ace076d422,5e5794daaa13f73e2f1cdb4114529843,delivered,2018-04-22 11:34:42,2018-04-24 19:04:46,2018-04-24 03:14:34,2018-04-27 20:55:28,2018-05-16 00:00:00,744bade1fcf9ff3f31d860ace076d422,2.0,credit_card,0,58.69
1,744bade1fcf9ff3f31d860ace076d422,5e5794daaa13f73e2f1cdb4114529843,delivered,2018-04-22 11:34:42,2018-04-24 19:04:46,2018-04-24 03:14:34,2018-04-27 20:55:28,2018-05-16 00:00:00,744bade1fcf9ff3f31d860ace076d422,2.0,credit_card,0,58.69
2,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018-05-15 16:25:14,2018-05-15 16:36:52,2018-05-17 12:37:00,2018-05-24 15:45:41,2018-06-06 00:00:00,1a57108394169c0b47d8f876acc9ba2d,2.0,credit_card,0,129.94
3,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018-05-15 16:25:14,2018-05-15 16:36:52,2018-05-17 12:37:00,2018-05-24 15:45:41,2018-06-06 00:00:00,1a57108394169c0b47d8f876acc9ba2d,2.0,credit_card,0,129.94


In [9]:
query = """
SELECT *
FROM orders o LEFT JOIN payments p ON (p.order_id=o.order_id)
WHERE p.order_id == '744bade1fcf9ff3f31d860ace076d422' OR p.order_id == '1a57108394169c0b47d8f876acc9ba2d'
"""

table = pd.read_sql_query( query, conn )
table

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,744bade1fcf9ff3f31d860ace076d422,5e5794daaa13f73e2f1cdb4114529843,delivered,2018-04-22 11:34:42,2018-04-24 19:04:46,2018-04-24 03:14:34,2018-04-27 20:55:28,2018-05-16 00:00:00,744bade1fcf9ff3f31d860ace076d422,2.0,credit_card,0,58.69
1,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018-05-15 16:25:14,2018-05-15 16:36:52,2018-05-17 12:37:00,2018-05-24 15:45:41,2018-06-06 00:00:00,1a57108394169c0b47d8f876acc9ba2d,2.0,credit_card,0,129.94
2,744bade1fcf9ff3f31d860ace076d422,5e5794daaa13f73e2f1cdb4114529843,delivered,2018-04-22 11:34:42,2018-04-24 19:04:46,2018-04-24 03:14:34,2018-04-27 20:55:28,2018-05-16 00:00:00,744bade1fcf9ff3f31d860ace076d422,2.0,credit_card,0,58.69
3,1a57108394169c0b47d8f876acc9ba2d,48ebb06cf56dba9d009230cc751bb195,delivered,2018-05-15 16:25:14,2018-05-15 16:36:52,2018-05-17 12:37:00,2018-05-24 15:45:41,2018-06-06 00:00:00,1a57108394169c0b47d8f876acc9ba2d,2.0,credit_card,0,129.94


**_It seem to be some kind of error then. So we should actually repeat the initial query:_**

In [12]:
query = """
SELECT 
	max( payment_installments ) ,
	min( payment_installments )
FROM payments p
WHERE payment_installments != 0
"""

table = pd.read_sql_query( query, conn )
table

,max( payment_installments ),min( payment_installments )
0,24,1


## What are the top 10 orders with the biggest payment values?

In [7]:
query = """
SELECT 
	order_id ,
	payment_value 
FROM payments p 
ORDER BY payment_value DESC 
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,order_id,payment_value
0,03caa2c082116e1d31e67e9ae3700499,13664.08
1,736e1922ae60d0d6a89247b851902527,7274.88
2,0812eb902a67711a1cb742b3cdaa65ae,6929.31
3,fefacc66af859508bf1a7934eab1e97f,6922.21
4,f5136e38d1a14a4dbd87dff67da82701,6726.66
5,2cc9089445046817a7539d90805e6e5a,6081.54
6,a96610ab360d42a2e5335a3998b4718a,4950.34
7,b4c4b76c642808cbe472a32b86cddc95,4809.44
8,199af31afc78c699f0dbf71fb178d4d4,4764.34
9,8dbc85d1447242f3b127dda390d56e19,4681.78


## What are the 10 last orders with the lowest payment values?

In [8]:
query = """
SELECT 
	order_id ,
	payment_value 
FROM payments p 
ORDER BY payment_value 
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,order_id,payment_value
0,8bcbe01d44d147f901cd3192671144db,0.00
1,fa65dad1b0e818e3ccc5cb0e39231352,0.00
2,6ccb433e00daae1283ccc956189c82ae,0.00
3,4637ca194b6387e2d538dc89b124b0ee,0.00
4,00b1cb0320190ca0daa2c88b35206009,0.00
5,45ed6e85398a87c253db47c2d9f48216,0.00
6,fa65dad1b0e818e3ccc5cb0e39231352,0.00
7,c8c528189310eaa44a745b8d9d26908b,0.00
8,b23878b3e8eb4d25a158f57d96331b18,0.00
9,7db5f2eb8f5f54db9f9e71ba4296bcbf,0.01


**_It may look a bit weird to have zero payment values, but we can check whether this is an error._**

**_It doesn't seem so._**

In [9]:
query = """
SELECT *
FROM payments p INNER JOIN orders o ON (o.order_id=p.order_id)
WHERE p.payment_value == 0
"""

table = pd.read_sql_query( query, conn )
table

,order_id,payment_sequential,payment_type,payment_installments,payment_value,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,8bcbe01d44d147f901cd3192671144db,4.0,voucher,1,0.0,8bcbe01d44d147f901cd3192671144db,f2def7f64f36952f2f5a9791f0285f34,delivered,2018-01-24 23:24:14,2018-01-24 23:37:20,2018-01-30 22:43:30,2018-02-02 21:26:42,2018-02-21 00:00:00
1,8bcbe01d44d147f901cd3192671144db,4.0,voucher,1,0.0,8bcbe01d44d147f901cd3192671144db,f2def7f64f36952f2f5a9791f0285f34,delivered,2018-01-24 23:24:14,2018-01-24 23:37:20,2018-01-30 22:43:30,2018-02-02 21:26:42,2018-02-21 00:00:00
2,fa65dad1b0e818e3ccc5cb0e39231352,14.0,voucher,1,0.0,fa65dad1b0e818e3ccc5cb0e39231352,9af2372a1e49340278e7c1ef8d749f34,shipped,2017-04-20 12:45:34,2017-04-22 09:10:13,2017-04-24 11:31:17,None,2017-05-18 00:00:00
3,fa65dad1b0e818e3ccc5cb0e39231352,14.0,voucher,1,0.0,fa65dad1b0e818e3ccc5cb0e39231352,9af2372a1e49340278e7c1ef8d749f34,shipped,2017-04-20 12:45:34,2017-04-22 09:10:13,2017-04-24 11:31:17,None,2017-05-18 00:00:00
4,6ccb433e00daae1283ccc956189c82ae,4.0,voucher,1,0.0,6ccb433e00daae1283ccc956189c82ae,843b211abe7b0264dd4a69eafc5bdf43,delivered,2017-10-26 23:51:35,2017-10-27 00:46:58,2017-10-27 19:53:03,2017-11-08 18:47:00,2017-11-22 00:00:00
5,6ccb433e00daae1283ccc956189c82ae,4.0,voucher,1,0.0,6ccb433e00daae1283ccc956189c82ae,843b211abe7b0264dd4a69eafc5bdf43,delivered,2017-10-26 23:51:35,2017-10-27 00:46:58,2017-10-27 19:53:03,2017-11-08 18:47:00,2017-11-22 00:00:00
6,4637ca194b6387e2d538dc89b124b0ee,1.0,not_defined,1,0.0,4637ca194b6387e2d538dc89b124b0ee,a73c1f73f5772cf801434bf984b0b1a7,canceled,2018-09-03 14:14:25,None,None,None,2018-09-10 00:00:00
7,4637ca194b6387e2d538dc89b124b0ee,1.0,not_defined,1,0.0,4637ca194b6387e2d538dc89b124b0ee,a73c1f73f5772cf801434bf984b0b1a7,canceled,2018-09-03 14:14:25,None,None,None,2018-09-10 00:00:00
8,00b1cb0320190ca0daa2c88b35206009,1.0,not_defined,1,0.0,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,None,None,None,2018-09-12 00:00:00
9,00b1cb0320190ca0daa2c88b35206009,1.0,not_defined,1,0.0,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,None,None,None,2018-09-12 00:00:00


## What is the average payment value for each of its types?

In [11]:
query = """
SELECT
	payment_type ,
	avg( payment_value )
FROM payments p 
GROUP BY payment_type 
"""

table = pd.read_sql_query( query, conn )
table

,payment_type,avg( payment_value )
0,boleto,145.034435
1,credit_card,163.319021
2,debit_card,142.570170
3,not_defined,0.000000
4,voucher,65.703354


**_Just checking the "not_defined" type. Maybe it could be an exchange coupon._**

In [12]:
query = """
SELECT *
FROM payments p 
WHERE payment_type == 'not_defined'
"""

table = pd.read_sql_query( query, conn )
table

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,4637ca194b6387e2d538dc89b124b0ee,1.0,not_defined,1,0.0
1,00b1cb0320190ca0daa2c88b35206009,1.0,not_defined,1,0.0
2,c8c528189310eaa44a745b8d9d26908b,1.0,not_defined,1,0.0


## What are the top 5 clients with the biggest "boleto" payment values?

In [18]:
query = """
SELECT 
	o.customer_id ,
	p.payment_value
FROM payments p INNER JOIN orders o ON (o.order_id = p.order_id)
WHERE p.payment_type == 'boleto'
ORDER BY p.payment_value DESC
LIMIT 5
"""

table = pd.read_sql_query( query, conn )
table

,customer_id,payment_value
0,ec5b2ba62e574342386871631fafd3fc,7274.88
1,ec5b2ba62e574342386871631fafd3fc,7274.88
2,f48d464a0baaea338cb25f816991ab1f,6922.21
3,f48d464a0baaea338cb25f816991ab1f,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66


**_Weird to have repeated lines. Let's check it._**

In [15]:
query = """
SELECT 
	DISTINCT o.customer_id ,
	p.payment_value
FROM payments p INNER JOIN orders o ON (o.order_id = p.order_id)
WHERE p.payment_type == 'boleto'
ORDER BY p.payment_value DESC
LIMIT 5
"""

table = pd.read_sql_query( query, conn )
table

,customer_id,payment_value
0,ec5b2ba62e574342386871631fafd3fc,7274.88
1,f48d464a0baaea338cb25f816991ab1f,6922.21
2,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66
3,05455dfa7cd02f13d132aa7a6a9729c6,6081.54
4,31e83c01fce824d0ff786fcd48dad009,3979.55


## What are the top 5 clients with the biggest credit card payment values?

In [20]:
query = """
SELECT 
	o.customer_id ,
	p.payment_value
FROM payments p INNER JOIN orders o ON (o.order_id = p.order_id)
WHERE p.payment_type == 'credit_card'
ORDER BY p.payment_value DESC
LIMIT 5
"""

table = pd.read_sql_query( query, conn )
table

,customer_id,payment_value
0,1617b1357756262bfa56ab541c47bc16,13664.08
1,1617b1357756262bfa56ab541c47bc16,13664.08
2,c6e2731c5b391845f6800c97401a43a9,6929.31
3,c6e2731c5b391845f6800c97401a43a9,6929.31
4,df55c14d1476a9a3467f131269c2477f,4950.34


In [19]:
query = """
SELECT 
	DISTINCT o.customer_id ,
	p.payment_value
FROM payments p INNER JOIN orders o ON (o.order_id = p.order_id)
WHERE p.payment_type == 'credit_card'
ORDER BY p.payment_value DESC
LIMIT 5
"""

table = pd.read_sql_query( query, conn )
table

,customer_id,payment_value
0,1617b1357756262bfa56ab541c47bc16,13664.08
1,c6e2731c5b391845f6800c97401a43a9,6929.31
2,df55c14d1476a9a3467f131269c2477f,4950.34
3,e0a2412720e9ea4f26c1ac985f6a7358,4809.44
4,24bbf5fd2f2e1b359ee7de94defc4a15,4764.34


## What are the top 10 most expensive products?

In [8]:
# Here the DISTINCT wouldn't be needed (just a fortunate occasion), but too much care isn't a bad thing 

query = """
SELECT 
	DISTINCT product_id ,
	price
FROM items i
ORDER BY price DESC
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,product_id,price
0,489ae2aa008f021502940f251d4cce7f,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,6499.00
3,a6492cc69376c469ab6f61d8f44de961,4799.00
4,c3ed642d592594bb648ff4a04cee2747,4690.00
5,259037a6a41845e455183f89c5035f18,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
8,dd113cb02b2af9c8e5787e8f1f0722f6,4059.00
9,6902c1962dd19d540807d0ab8fade5c6,3999.90


In [21]:
# Indeed,

query = """
SELECT 
	product_id ,
	price
FROM items i
ORDER BY price DESC
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,product_id,price
0,489ae2aa008f021502940f251d4cce7f,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,6499.00
3,a6492cc69376c469ab6f61d8f44de961,4799.00
4,c3ed642d592594bb648ff4a04cee2747,4690.00
5,259037a6a41845e455183f89c5035f18,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
8,dd113cb02b2af9c8e5787e8f1f0722f6,4059.00
9,6902c1962dd19d540807d0ab8fade5c6,3999.90


## What are the top 10 most cheap products?

In [6]:
# Here the DISTINCT is needed (see the next snippet). 
# This makes sense, since cheap products might be bought more oftenly than expensive ones

query = """
SELECT 
	product_id ,
	price
FROM items i
ORDER BY price
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,product_id,price
0,8a3254bee785a526d548a81a9bc3c9be,0.85
1,8a3254bee785a526d548a81a9bc3c9be,0.85
2,8a3254bee785a526d548a81a9bc3c9be,0.85
3,270516a3f41dc035aa87d220228f844c,1.20
4,05b515fdc76e888aada3c6d66c201dff,1.20
5,05b515fdc76e888aada3c6d66c201dff,1.20
6,05b515fdc76e888aada3c6d66c201dff,1.20
7,05b515fdc76e888aada3c6d66c201dff,1.20
8,05b515fdc76e888aada3c6d66c201dff,1.20
9,05b515fdc76e888aada3c6d66c201dff,1.20


In [7]:
query = """
SELECT 
	DISTINCT product_id ,
	price
FROM items i
ORDER BY price
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,product_id,price
0,8a3254bee785a526d548a81a9bc3c9be,0.85
1,270516a3f41dc035aa87d220228f844c,1.20
2,05b515fdc76e888aada3c6d66c201dff,1.20
3,46fce52cef5caa7cc225a5531c946c8b,2.20
4,310dc32058903b6416c71faff132df9e,2.29
5,680cc8535be7cc69544238c1d6a83fe8,2.90
6,2e8316b31db34314f393806fd7b6e185,2.99
7,44d53f1240d6332232e4393c06500475,3.00
8,29781581fb82fe2389560a3a5331d0ee,3.06
9,1716ea399ed8ee62ba811e6f55180f45,3.49


## What are the top 10 most bought categories?

In [10]:
query = """
SELECT 
	p.product_category_name ,
	COUNT( i.product_id ) AS qty_bought
FROM items i INNER JOIN products p ON (p.product_id=i.product_id)
GROUP BY p.product_category_name 
ORDER BY qty_bought DESC 
LIMIT 10
"""

table = pd.read_sql_query( query, conn )
table

,product_category_name,qty_bought
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


## What are the top 5 products with the biggest number of reviews?

In [5]:
query = """
SELECT 
	i.product_id ,
	count( r.review_score ) as number_of_review_scores
FROM items i LEFT JOIN reviews r ON (r.order_id=i.order_id)
GROUP BY i.product_id 
ORDER BY number_of_review_scores DESC 
LIMIT 5 
"""

table = pd.read_sql_query( query, conn )
table

,product_id,number_of_review_scores
0,aca2eb7d00ea1a7b8ebd4e68314663af,527
1,99a4788cb24856965c36a24e339b6058,491
2,422879e10f46682990de24d770e7f83d,487
3,389d119b48cf3043d311335e499d9c6b,392
4,368c6c730842d78016ad823897a372db,391


**_Notice every order has a review score:_**

In [7]:
query = """
SELECT * 
FROM items i LEFT JOIN reviews r ON (r.order_id=i.order_id) 
WHERE r.review_score IS NULL
"""

pd.read_sql_query( query, conn )

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp


**_But not every review has a comment besides the score_**

In [8]:
query = """
SELECT 
	i.product_id ,
	count( r.review_comment_message ) as number_of_review_comments
FROM items i LEFT JOIN reviews r ON (r.order_id=i.order_id)
GROUP BY i.product_id 
ORDER BY number_of_review_comments DESC 
LIMIT 5
"""

table = pd.read_sql_query( query, conn )
table

,product_id,number_of_review_comments
0,99a4788cb24856965c36a24e339b6058,246
1,422879e10f46682990de24d770e7f83d,230
2,368c6c730842d78016ad823897a372db,212
3,53759a2ecddad2bb87a079a1f1519f73,198
4,aca2eb7d00ea1a7b8ebd4e68314663af,197


## What are the top 10 products without any reviews?

**_As we saw above, every order has a review score, but not every score is accompanied by a review comment. Let's seek those products with the least comments then._**

In [10]:
query = """
SELECT 
	i.product_id ,
	count( i.product_id ) AS no_rev_comments_count
FROM items i LEFT JOIN reviews r ON (r.order_id=i.order_id)
WHERE r.review_comment_message IS NULL
GROUP BY product_id
ORDER BY no_rev_comments_count DESC 
LIMIT 10
"""

pd.read_sql_query( query, conn )

,product_id,no_rev_comments_count
0,aca2eb7d00ea1a7b8ebd4e68314663af,330
1,422879e10f46682990de24d770e7f83d,257
2,99a4788cb24856965c36a24e339b6058,245
3,53b36df67ebb7c41585e8d54d6772e08,206
4,389d119b48cf3043d311335e499d9c6b,197
5,d1c427060a0f73f6b889a5c7c61f2ac4,195
6,368c6c730842d78016ad823897a372db,179
7,53759a2ecddad2bb87a079a1f1519f73,177
8,154e7e31ebfa092203795c972e5804a6,175
9,3dd2a17168ec895c781a9191c1e95ad7,162


## What are the top 10 clients with the biggest number of orders?

In [8]:
query = """
SELECT 
	customer_id ,
	count( order_id ) AS orders_count
FROM orders o 
GROUP BY customer_id
ORDER BY orders_count
LIMIT 10
"""

pd.read_sql_query( query, conn )

,customer_id,orders_count
0,00012a2ce6f8dcda20d059ce98491703,2
1,000161a058600d5901f007fab4c27140,2
2,0001fd6190edaaf884bcaf3d49edf079,2
3,0002414f95344307404f0ace7a26f1d5,2
4,000379cdec625522490c315e70c7a9fb,2
5,0004164d20a9e969af783496f3408652,2
6,000419c5494106c306a97b5635748086,2
7,00046a560d407e99b969756e0b10f282,2
8,00050bf6e01e69d5c0fd612f1bcfb69c,2
9,000598caf2ef4117407665ac33275130,2


**_Seems a bit weird that every of them has only 2 orders. Let's investigate this._**

In [9]:
query = """
WITH o_count_sub AS (
	SELECT 
		customer_id ,
		count( order_id ) AS orders_count
	FROM orders o 
	GROUP BY customer_id 
)
SELECT *
FROM o_count_sub ocs 
WHERE orders_count != 2
"""

pd.read_sql_query( query, conn )

,customer_id,orders_count


**_This means every order per customer is recorded as a duplicate. Moreover, it seems the customer_id field changes everytime, independently of it being the same customer on a different visit (this is a guess)._**

**_Below, let's double-check this._**

In [10]:
query = """
SELECT
	count( customer_id ) AS  num_custom ,
	count( DISTINCT customer_id ) AS  dist_num_custom
FROM orders o
"""

pd.read_sql_query( query, conn )

,num_custom,dist_num_custom
0,198882,99441


**_Notice that the first column is precisely the double of the second._**

In [16]:
198882/99441

2.0

## What are the top 10 clients with the lowest number of orders?

**_See discussion above_**

## What sellers are there in the database?

In [11]:
query = """
SELECT
	s.seller_id 
FROM sellers s
"""

pd.read_sql_query( query, conn )

,seller_id
0,3442f8959a84dea7ee197c632cb2df15
1,d1b65fc7debc3361ea86b5f14c68d2e2
2,ce3ad9de960102d0677a81f5d0bb7b2d
3,c0f3eea2e14555b6faeea3dd58c1b1c3
4,51a04a8a6bdcb23deccc82b0b80742cf
...,...
3090,98dddbc4601dd4443ca174359b237166
3091,f8201cab383e484733266d1906e2fdfa
3092,74871d19219c7d518d0090283e03c137
3093,e603cf3fec55f8697c9059638d6c8eb5


## What is the distribution of sellers by state?

In [12]:
query = """
SELECT
	seller_state ,
	count( seller_id ) as num_of_sellers
FROM sellers s 
GROUP BY seller_state 
ORDER BY num_of_sellers DESC
"""

pd.read_sql_query( query, conn )

,seller_state,num_of_sellers
0,SP,1849
1,PR,349
2,MG,244
3,SC,190
4,RJ,171
5,RS,129
6,GO,40
7,DF,30
8,ES,23
9,BA,19


## What is the distribution of clients by state?

In [14]:
query = """
SELECT 
	customer_state ,
	count( customer_id ) AS num_of_customers
FROM customer c 
GROUP BY customer_state 
ORDER BY num_of_customers DESC
"""

pd.read_sql_query( query, conn )

,customer_state,num_of_customers
0,SP,83492
1,RJ,25704
2,MG,23270
3,RS,10932
4,PR,10090
5,SC,7274
6,BA,6760
7,DF,4280
8,ES,4066
9,GO,4040


## What are the top 10 sellers that receveid the most "boleto" payments?

In [14]:
query = """
SELECT 
	s.seller_id ,
	count( p.payment_type ) AS num_of_boleto_payments
FROM items i 
	LEFT JOIN payments p ON (i.order_id=p.order_id)
	INNER JOIN sellers s ON (i.seller_id=s.seller_id)
WHERE p.payment_type == 'boleto'
GROUP BY s.seller_id 
ORDER BY num_of_boleto_payments DESC 
LIMIT 10
"""

pd.read_sql_query( query, conn )

,seller_id,num_of_boleto_payments
0,1f50f920176fa81dab994f9023523100,541
1,6560211a19b47992c3666cc44a7e94c0,419
2,cc419e0650a3c5ba77189a1882b7556a,395
3,955fee9216a65b617aa5c0531780ce60,386
4,7c67e1448b00f6e969d365cea6b010ab,363
5,4a3ca9315b744ce9f8e9374361493884,338
6,da8622b14eb17ae2831f4ac5b9dab84a,329
7,1025f0e2d44d7041d6cf58b6550e0bfa,289
8,8b321bb669392f5163d04c59e235e066,255
9,7a67c85e85bb2ce8582c35f2203ad736,235


## What are 10 worst sellers, in terms of number of orders?

**_Recall that the 'items' table has the 'order_id' feature repeated over the rows for purchases with more than 1 item, so we have to specify 'DISTINCT' inside 'count()'_**

In [17]:
query = """
SELECT 
	s.seller_id ,
	count( DISTINCT i.order_id ) AS num_of_orders
FROM items i
	INNER JOIN sellers s ON (i.seller_id=s.seller_id) 
GROUP BY s.seller_id 
ORDER BY num_of_orders
"""

pd.read_sql_query( query, conn )

,seller_id,num_of_orders
0,001e6ad469a905060d959994f1b41e4f,1
1,003554e2dce176b5555353e4f3555ac8,1
2,00ab3eff1b5192e5f1a63bcecfee11c8,1
3,00d8b143d12632bad99c0ad66ad52825,1
4,010da0602d7774602cd1b3f5fb7b709e,1
...,...,...
3090,da8622b14eb17ae2831f4ac5b9dab84a,1314
3091,1f50f920176fa81dab994f9023523100,1404
3092,cc419e0650a3c5ba77189a1882b7556a,1706
3093,4a3ca9315b744ce9f8e9374361493884,1806


## How many products are bought, in average, by order?

In [20]:
query = """
WITH items_count AS (
	SELECT
		order_id ,
		COUNT( order_item_id ) AS num_of_items 
	FROM items i 
	GROUP BY order_id
)
SELECT AVG( num_of_items ) 
FROM items_count
"""

pd.read_sql_query( query, conn )

,AVG( num_of_items )
0,1.141731
